In [2]:
wdir="AM_videogames/"

import pandas as pd
import numpy as np
import time
import spacy
import spacy_transformers
import pickle
# sp_trf_vector = spacy.load('en_core_web_trf', disable[])
# spacy.require_gpu()
sp_trf = spacy.load('en_core_web_trf')

sp_trf.add_pipe('sentencizer')

# https://applied-language-technology.readthedocs.io/en/latest/notebooks/part_iii/04_embeddings_continued.html

In [3]:
!nvcc --version

'nvcc' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [4]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.1.1                         
Location         E:\Anaconda\envs\BasilTFE2\lib\site-packages\spacy
Platform         Windows-10-10.0.19043-SP0     
Python version   3.8.11                        
Pipelines        en_core_web_md (3.1.0), en_core_web_trf (3.1.0)



In [3]:
# POSLemmaChoice="SP" # WN TB SP
wdir="AM_videogames/"
chosen_game="diablo III"
chosen_game_list=[]
used_games=["Company of Heroes 2",      ## games with most reviews ( around 100 each ) in the dataset     
"Diablo III",                     
"Sid Meier's Civilization VI",   
"SUPERHOT",                       
"XCOM 2"]  

basic_df = pd.read_pickle(wdir+"1_df.pkl") # 2 is sentence split, with stopwords 
basic_df=basic_df.loc[basic_df["game_title"]==chosen_game]
basic_df=basic_df.reset_index(drop=True)
basic_df.head(10)
texts=[]
for index,row in basic_df.iterrows():
    texts.append(row["review_text"])

In [4]:
%%timeit -n 1 -r 1 # number of loop
i=0
doc_list=[]
texts_number=len(texts)
for doc in sp_trf.pipe(texts, batch_size=64):
    i+=1
    print(int(i/texts_number*100) , " % processed", end="\r", flush=True)
    doc_list.append(doc)

''' Serialization '''
# Serialize vocab (actually the whole NLP ojbect)
pickle.dump(doc_list, open(wdir+chosen_game+"_SP_trf.pickle", "wb"))

''' Deserialization '''
loaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP_trf.pickle", "rb"))
print(len(loaded_doc_list))

593  % processed
3min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
sp_trf.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x1eea9143180>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1eea8e89d10>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1eea9147340>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1eea912b9c0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1eea912a740>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1eea8f90b80>),
 ('sentencizer', <spacy.pipeline.sentencizer.Sentencizer at 0x1eea9166c80>)]

In [6]:
''' Deserialization '''
loaded_doc_list = pickle.load(open(wdir+chosen_game+"_SP_trf.pickle", "rb"))
print(len(loaded_doc_list))


593


In [65]:
# token attributes: https://spacy.io/api/token

x=0
print(loaded_doc_list[x][0].sent)
print(loaded_doc_list[x][0].pos_,loaded_doc_list[x][1].pos_) # pos pour fop extraction 
print(loaded_doc_list[x][0].lemma_,loaded_doc_list[x][1].lemma_) # lemma pour fop extraction 
print(loaded_doc_list[x][0].is_stop,loaded_doc_list[x][1].is_stop) # pour ignorer les stop_words, possibilité d'agrandir la liste 
print(loaded_doc_list[x]._.trf_data.tensors[0].shape) # osef des vecteurs askip finalement 
print(loaded_doc_list[x].__len__()) # nbre de token
# print(loaded_doc_list[x]._.trf_data.tokens['input_texts'])
print(len(loaded_doc_list))


My title says all.
PRON NOUN
my title
True False
(2, 140, 768)
160
593


In [108]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half]

def tag_convert(tag):
    if tag == "NOUN":
        return 'n'
    if tag == "VERB":
        return 'v'
    if tag == "ADJ":
        return 'a'
    if tag == "ADV":
        return 'r'
    

In [109]:
data = []
index = 0
for doc in loaded_doc_list:
    for token in doc:
        if not token.is_stop and (token.pos_ == "NOUN" or token.pos_ == "ADJ" or token.pos_ == "ADV" or token.pos_ == "VERB"):
            tokenlist = []
            tokenlist.append(index)
            tokenlist.append(chosen_game)
            tokenlist.append(token.sent.text)
            tokenlist.append([(token.lemma_, tag_convert(token.pos_))])
            tokenlist.append('blabla')
            data.append(tokenlist)
        index +=1
FOPs_df = pd.DataFrame(data, columns = ['review_id','game_title', 'sentence', 'tags', 'processed_sentence'])
aggregation_functions = {'review_id':'first', 'game_title':'first', 'tags':'sum', 'sentence':'first', 'processed_sentence':'first'}
FOPs_df_new = FOPs_df.groupby(FOPs_df['sentence']).aggregate(aggregation_functions)
FOPs_df_new['tags'] = FOPs_df_new['tags'].apply(split_list)

In [114]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sentiment_polarity import * 
from FOPs import * 
FOPs_df_new['vader_polarity'] = FOPs_df_new['sentence'].apply(getVaderPolarity)

chosen_game="diablo III"

FOPs_game_dic = extract_all_FOPs(FOPs_df_new, chosen_game)

print(FOPs_game_dic)

save_FOPs_dic_csv(FOPs_game_dic, wdir,chosen_game+"_trl")

2370  sentences for the game  diablo III
{'conclusion': {'fun': {'count': 1, 'importance': 0.7818}, 'satisfying': {'count': 1, 'importance': 0.7818}, 'go': {'count': 1, 'importance': 0.7818}, 'come': {'count': 2, 'importance': 0.7818}}, 'game': {'fun': {'count': 72, 'importance': 27.716300000000004}, 'satisfying': {'count': 1, 'importance': 0.7818}, 'go': {'count': 37, 'importance': 12.785800000000004}, 'come': {'count': 33, 'importance': 5.3833}, 'high': {'count': 16, 'importance': 1.3842999999999996}, 'hack': {'count': 6, 'importance': -0.9671}, 'start': {'count': 24, 'importance': 2.390999999999999}, 'mindless': {'count': 4, 'importance': -0.25280000000000014}, 'pick': {'count': 7, 'importance': 0.4525999999999999}, 'especially': {'count': 8, 'importance': -0.8071}, 'probably': {'count': 12, 'importance': 3.6457}, 'random': {'count': 8, 'importance': 0.16019999999999998}, 'want': {'count': 37, 'importance': 9.672099999999999}, 'replay': {'count': 5, 'importance': 0.3053}, 'decide': 

In [1]:
FOPs_df = pd.read_csv(wdir+chosen_game+"_trl_FOPs.csv",error_bad_lines=False)  # Creation of the dataframe 

FOPs_df["count"]=FOPs_df["count"]/2
FOPs_df["importance"]=FOPs_df["importance"]/2
FOPs_df["polarity"] = FOPs_df["importance"]/FOPs_df["count"]
FOPs_df["importance"] = FOPs_df["importance"]/FOPs_df["importance"].max()
FOPs_df["importance"]= abs(FOPs_df["importance"])
FOPs_df["FOP"]= (FOPs_df["feature"]) + " " + (FOPs_df["opinion"])

FOPs_df=FOPs_df.reset_index(drop=True)
# maxCount = FOPs_df.iloc[0]["count"]
# FOPs_df = FOPs_df.loc[FOPs_df["count"] > maxCount/20]
# FOPs_df = FOPs_df.sort_values('importance', ascending=False)
print(FOPs_df.shape)
FOPs_df.head(20)

NameError: name 'pd' is not defined

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [5, 20]

FOPs_df = FOPs_df.sort_values('importance', ascending=False)
FOPs_df_top100 = FOPs_df.head(100)

ax = FOPs_df_top100.plot.barh(x='FOP', y = 'importance')
ax.invert_yaxis()
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [5, 20]

FOPs_df = FOPs_df.sort_values('count', ascending=False)
FOPs_df_top100 = FOPs_df.head(100)

ax = FOPs_df_top100.plot.barh(x='FOP', y = 'count')
ax.invert_yaxis()
plt.show()

In [ ]:
aggregation_functions = {'count':'sum', 'importance':'sum', 'feature':'first'}
FOPs_df_new = FOPs_df.groupby(FOPs_df['feature']).aggregate(aggregation_functions)

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [5, 20]

FOPs_df_new = FOPs_df_new.sort_values('count', ascending=False)
FOPs_df_new_top100 = FOPs_df_new.head(100)

ax = FOPs_df_new_top100.plot.barh(x='feature', y = 'count')
ax.invert_yaxis()
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [5, 20]

FOPs_df_new = FOPs_df_new.sort_values('importance', ascending=False)
FOPs_df_new_top100 = FOPs_df_new.head(100)

ax = FOPs_df_new_top100.plot.barh(x='feature', y = 'importance')
ax.invert_yaxis()
plt.show()